In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'/Users/lakshitjain/Developer/ML Projects/Coccidiosis-Disease-Classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    checkpoint_model_filepath: Path
    tensorboard_root_log_dir: Path

In [4]:
from coccidiosisDiseaseClassification.constants import *
from coccidiosisDiseaseClassification.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config=self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir), Path(config.tensorboard_root_log_dir)])
        
        return PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir)
        )

In [5]:
# Component
import os 
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [6]:
class PrepareCallback:
    def __init__(self,config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y%m%d-%H%M%S")
        tb_running_log_dir=os.path.join(self.config.tensorboard_root_log_dir, f"tb_logs_at_{timestamp}")
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [7]:
# Pipeline
try:
    config=ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e


[2025-08-21 21:49:41,052] : INFO : common : YAML file config/config.yaml loaded successfully.
[2025-08-21 21:49:41,055] : INFO : common : YAML file params.yaml loaded successfully.
[2025-08-21 21:49:41,057] : INFO : common : Created directory: artifacts
[2025-08-21 21:49:41,058] : INFO : common : Created directory: artifacts/prepare_callbacks/checkpoint_dir
[2025-08-21 21:49:41,059] : INFO : common : Created directory: artifacts/prepare_callbacks/tensorboard_log_dir
